In [2]:
import pandas as pd
from neo4j import GraphDatabase

pd.set_option('display.max_columns',None)

df = pd.read_csv('./data/restaurant_reviews_withsentiment.csv')
df_loc = pd.read_csv('./data/restaurants_latest.csv')

In [3]:
df.head(100)

,Title,Body,Reviewer,Reviewer_Level,Reviewer_NumReviews,ReviewDateTime,Restaurant,Review,Food,RestaurantReview,FoodReview,RestaurantFeature(spacy),FoodFeature(spacy),RestaurantFeature(online),FoodFeature(online),RestaurantSentimentProba,FoodSentimentProba,OverallSentimentClass
0,Glad It’s Back!,We ordered two appetisers and four main dishes...,Sofina Ng,3.0,9.0,10m ago,/babasan-by-uncle-kiisu,Glad It’s Back!. We ordered two appetisers and...,"beer,fruit",glad it’s back!. we ordered two appetisers and...,kronenbourg blanc beer was very enjoyable too....,"ordered,main,unique,boasting,manifested,creati...","enjoyable,sweet,mild",unique,"enjoyable,fruity,mild,strong,sweet",0.981540,0.805150,1
1,Yuzu Ochazuke ($24),This rice in soup dish is the first one I’ve h...,Jan L,3.0,10.0,5d ago,/babasan-by-uncle-kiisu,Yuzu Ochazuke ($24). This rice in soup dish is...,rice,yuzu ochazuke ($24). boy did it impress! perfe...,this rice in soup dish is the first one i’ve h...,"impress,perfect,harmonious,fresh,generous,serv...","great,good","first,good,great,style","fresh,overpowering",0.963360,0.993274,1
2,Belly Roll Mee ($15),"The successor to Uncle Kiisu, Babasan’s menu d...",Dex Neo,8.0,958.0,Oct 24 at 5:42pm,/babasan-by-uncle-kiisu,Belly Roll Mee ($15). The successor to Uncle K...,"crispy pork,mee siam,pork,pork belly",belly roll mee ($15). the successor to uncle k...,i was seriously wondering how they would pull ...,"dreamt,good,sounds,heavy,provided,cut,amazingl...","seriously wondering,pull,crispy,stuffed,tender...","affordable,good,heavy","crispy,heavy,icy,rich,spicy,stuffed,tangy,tast...",0.986915,0.959834,1
3,Samurai Kueh Pie Tee,This was like a little bomb that exploded with...,Komal Salve,7.0,208.0,Oct 23 at 10:06am,/babasan-by-uncle-kiisu,Samurai Kueh Pie Tee. This was like a little b...,"fish,fish roe,rice,roe,seafood,unagi",tee. this was like a little bomb that exploded...,samurai kueh pie i'm not a big fan of the seaf...,"little,exploded,eating,try,super worth","big,loved,flying,smooth","big,little","smooth,texture",0.959460,0.964552,1
4,Build Your Own Bowl (Small),Most salad/grain bowls serve up similar stuff ...,Nom Nom,3.0,15.0,Oct 25 at 10:13pm,/yobo,Build Your Own Bowl (Small). Most salad/grain ...,"brown rice,cream,pumpkin,rice,salad",build your own bowl (small). but i really enjo...,most salad/grain bowls serve up similar stuff ...,"build,small,enjoyed,served","salad,serve,similar,creamy,paired,brown,roaste...","good,small","creamy,roasted",0.915978,0.972140,1
5,Healthy Warm Salad Bowls At Affordable Pricing,"I ordered 2 servings of honey soy, 1 serving o...",Ray L,2.0,1.0,Oct 18 at 3:40pm,/yobo,Healthy Warm Salad Bowls At Affordable Pricing...,"aglio olio,brussel sprout,fusilli,mushroom,nut...",this was all done using their online portal an...,healthy warm salad bowls at affordable pricing...,"online,paid,nice,late,eat,definitely recommend","healthy,warm,salad,affordable,ordered,serving,...","affordable,long,nice,warm","garlicky,healthy,light,nutty,strong,warm",0.925739,0.959115,1
6,For Delicious Build-Your-Own-Bowls in the CBD,Make note of this spot on Cecil Street for qui...,Burpple Guides,NaN,NaN,Oct 16 at 4:16pm,/yobo,For Delicious Build-Your-Own-Bowls in the CBD....,"brown rice,chicken,cream,kale chip,rice",for delicious build-your-own-bowls in the cbd....,keep it healthy with japanese brown rice as yo...,"delicious,build,quick,customisable,small,inclu...","healthy,japanese,brown,recommends,adding,cream...","japanese,small,top","bitter,creamy,delicious,earthy,healthy,hot,smo...",0.981325,0.928836,1
7,YOBO Grain Bowl,Newly opened grain bowl place located on the 2...,Gan Zhang Xun,6.0,179.0,Oct 8 at 1:51pm,/yobo,YOBO Grain Bowl. Newly opened grain bowl place...,"chicken,cream,kale chip,pumpkin,soy,soy chicken",yobo grain bowl. newly opened grain bowl place...,but the honey soy chicken could’ve been better...,"newly opened,located,2nd,ongoing,baked,crusted...","better seasoned,running,low,ran,super creamy,s...","better,e

In [4]:
df_loc.head()

,Unnamed: 0,Title,Link,Location,Latitude,Longitude,NumReviews,PriceRange,Tags,Categories,Address
0,0,Babasan by Uncle Kiisu,/babasan-by-uncle-kiisu,Clarke Quay,1.286034,103.849552,4,~$30/pax,['BEYOND'],"['Local Delights', ' Japanese', ' 1-for-1 Deal...",50 Circular Road Singapore 049405
1,1,Enjoy Eating House and Bar,/enjoy-eating-house-and-bar,Farrer Park,1.313191,103.859697,131,~$15/pax,"['BEYOND', 'Featured In 3 Guides']","['Bars', ' Local Delights', ' Zi Char', ' Thai...",383 Jalan Besar Kam Leng Hotel Singapore 209001
2,2,Wu Pao Chun,/wu-pao-chun-bakery,City Hall,1.292939,103.851480,79,~$13/pax,"['BEYOND', 'Featured In 1 Guide']","['Bread', ' 1-for-1 Deals', ' Burpple Beyond']",13 Stamford Road #01-19/20 Capitol Singapore ...
3,3,The Salted Plum (Circular Road),/thesaltedplumsg,Raffles Place,1.287783,103.848887,215,~$20/pax,"['BEYOND', 'Hot 100 2019']","['Taiwanese', ' Cheap & Good', ' 1-for-1 Deals...",10 Circular Road Singapore 049366
4,4,Rakki Bowl,/rakki-bowl,Tanjong Pagar,1.278173,103.843013,293,~$25/pax,"['BEYOND', 'Hot 100 2019', 'Featured In 2 Guid...","['Chirashi', ' Japanese', ' 1-for-1 Deals']",72 Duxton Road Singapore 089531


In [5]:
df_loc.rename(columns={'Link':'Restaurant'},inplace=True)
df = df.merge(df_loc[['Restaurant','Address']], how='left', on='Restaurant')

In [6]:
df['Review'] = df['Title'] + ":" + df['Body']

In [7]:
df['Food'].fillna("",inplace=True)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35791 entries, 0 to 35790
Data columns (total 19 columns):
Title                        35791 non-null object
Body                         35586 non-null object
Reviewer                     35791 non-null object
Reviewer_Level               35144 non-null float64
Reviewer_NumReviews          35144 non-null float64
ReviewDateTime               35791 non-null object
Restaurant                   35791 non-null object
Review                       35586 non-null object
Food                         35791 non-null object
RestaurantReview             35791 non-null object
FoodReview                   35791 non-null object
RestaurantFeature(spacy)     32867 non-null object
FoodFeature(spacy)           35274 non-null object
RestaurantFeature(online)    35791 non-null object
FoodFeature(online)          35791 non-null object
RestaurantSentimentProba     35791 non-null float64
FoodSentimentProba           35791 non-null float64
OverallSentimentClas

In [9]:
df.head()

,Title,Body,Reviewer,Reviewer_Level,Reviewer_NumReviews,ReviewDateTime,Restaurant,Review,Food,RestaurantReview,FoodReview,RestaurantFeature(spacy),FoodFeature(spacy),RestaurantFeature(online),FoodFeature(online),RestaurantSentimentProba,FoodSentimentProba,OverallSentimentClass,Address
0,Glad It’s Back!,We ordered two appetisers and four main dishes...,Sofina Ng,3.0,9.0,10m ago,/babasan-by-uncle-kiisu,Glad It’s Back!:We ordered two appetisers and ...,"beer,fruit",glad it’s back!. we ordered two appetisers and...,kronenbourg blanc beer was very enjoyable too....,"ordered,main,unique,boasting,manifested,creati...","enjoyable,sweet,mild",unique,"enjoyable,fruity,mild,strong,sweet",0.981540,0.805150,1,50 Circular Road Singapore 049405
1,Yuzu Ochazuke ($24),This rice in soup dish is the first one I’ve h...,Jan L,3.0,10.0,5d ago,/babasan-by-uncle-kiisu,Yuzu Ochazuke ($24):This rice in soup dish is ...,rice,yuzu ochazuke ($24). boy did it impress! perfe...,this rice in soup dish is the first one i’ve h...,"impress,perfect,harmonious,fresh,generous,serv...","great,good","first,good,great,style","fresh,overpowering",0.963360,0.993274,1,50 Circular Road Singapore 049405
2,Belly Roll Mee ($15),"The successor to Uncle Kiisu, Babasan’s menu d...",Dex Neo,8.0,958.0,Oct 24 at 5:42pm,/babasan-by-uncle-kiisu,Belly Roll Mee ($15):The successor to Uncle Ki...,"crispy pork,mee siam,pork,pork belly",belly roll mee ($15). the successor to uncle k...,i was seriously wondering how they would pull ...,"dreamt,good,sounds,heavy,provided,cut,amazingl...","seriously wondering,pull,crispy,stuffed,tender...","affordable,good,heavy","crispy,heavy,icy,rich,spicy,stuffed,tangy,tast...",0.986915,0.959834,1,50 Circular Road Singapore 049405
3,Samurai Kueh Pie Tee,This was like a little bomb that exploded with...,Komal Salve,7.0,208.0,Oct 23 at 10:06am,/babasan-by-uncle-kiisu,Samurai Kueh Pie Tee:This was like a little bo...,"fish,fish roe,rice,roe,seafood,unagi",tee. this was like a little bomb that exploded...,samurai kueh pie i'm not a big fan of the seaf...,"little,exploded,eating,try,super worth","big,loved,flying,smooth","big,little","smooth,texture",0.959460,0.964552,1,50 Circular Road Singapore 049405
4,Build Your Own Bowl (Small),Most salad/grain bowls serve up similar stuff ...,Nom Nom,3.0,15.0,Oct 25 at 10:13pm,/yobo,Build Your Own Bowl (Small):Most salad/grain b...,"brown rice,cream,pumpkin,rice,salad",build your own bowl (small). but i really enjo...,most salad/grain bowls serve up similar stuff ...,"build,small,enjoyed,served","salad,serve,similar,creamy,paired,brown,roaste...","good,small","creamy,roasted",0.915978,0.972140,1,182 Cecil Street #02-14 Frasers Tower Singapor...


In [10]:
from py2neo import Graph, Node, Relationship
# Input the user name and password set in the neo4j graph
graph = Graph("bolt://localhost:7687", auth=("neo4j","1234"))
graph.delete_all()

In [11]:
for index, row in df.iterrows():
    
    # Create notes for review, reviewer, restaurant, food review and restaurant review
    review = Node('Review', 
                  review = row['Review'],
                  overall_sentiment = row['OverallSentimentClass']) 

    reviewer = Node('Reviewer', 
                    name = row['Reviewer'],
                    reviewer_level = row['Reviewer_Level'], 
                    no_of_reviews= row['Reviewer_NumReviews'])
    
    restaurant = Node('Restaurant', 
                  name = row['Restaurant'],
                  address= row['Address'])
    
    foodreview = Node('Food Review', 
                      food_review = row['FoodReview'], 
                      food_sentiment = row['FoodSentimentProba'],
                      features = row['FoodFeature(online)'])
    
    restaurantreview = Node('Restaurant Review', 
                            restaurant_review = row['RestaurantReview'], 
                            restaurant_sentiment = row['RestaurantSentimentProba'], 
                            features = row['RestaurantFeature(online)'])
    
    # Remove duplicates for reviewer and restaurant
    graph.create(review)
    graph.merge(reviewer, 'Reviewer','name')
    graph.merge(restaurant, 'Restaurant', 'name')

    # for each food in review, create a node
    temp_food = row['Food'].split()
    for food in temp_food:
        if "," in food:
            food = food.split(",")
            for food in set(food): 
                food_node = Node('Food', 
                            name = food, 
                            features = row['FoodFeature(online)'])
                
                # Remove duplicates for food
                graph.merge(food_node,'Food','name')
                
                # Create relationships for review [contains]-> food, food [servedby]-> restaurant
                review_contains_food = Relationship(review, "Contains", food_node,
                   food_sentiment = row['FoodSentimentProba'],
                   features=row['FoodFeature(online)'])
                
                food_servedby_restaurant = Relationship(food_node, "Servedby", restaurant)
                
                graph.create(review_contains_food)
                graph.create(food_servedby_restaurant)
                
        else:
            food_node = Node('Food', 
                            name = food, 
                            features = row['FoodFeature(online)'])
                      
            graph.merge(food_node,'Food','name')

            review_contains_food = Relationship(review, "Contains", food_node,
                            food_sentiment = row['FoodSentimentProba'],
                            features=row['FoodFeature(online)'])

            food_servedby_restaurant = Relationship(food_node, "Servedby", restaurant)
            
            graph.create(review_contains_food)
            graph.create(food_servedby_restaurant)
                
    # Create relationships for reviewer [wrote]-> review, review [about]-> restaurant
    reviewer_wrote_review = Relationship(reviewer, "Wrote", review)
    
    review_about_restaurant = Relationship(review, "About", restaurant, 
                                           restaurant_sentiment = row['RestaurantSentimentProba'], 
                                           features=row['RestaurantFeature(online)'])
    
    graph.create(reviewer_wrote_review)
    graph.create(review_about_restaurant)   

In [16]:
output = pd.DataFrame(graph.run("MATCH (n) RETURN n"))#.to_table()

In [17]:
output.to_csv('./data/graph_output.csv')

In [18]:
output.shape

(41652, 1)